# Пример распознавание русской речи на обученной модели

In [1]:
import tensorflow as tf

import numpy as np
import os

from IPython.display import Audio


import scipy.io.wavfile as wav
from python_speech_features import fbank, mfcc
from keras.layers import LSTM, Dense, Convolution1D
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed, Bidirectional



Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
vocabulary = { 'а': 1,
               'б': 2,
               'в': 3,
               'г': 4,
               'д': 5,
               'е': 6,
               'ё': 7,
               'ж': 8,
               'з': 9,
               'и': 10,
               'й': 11,
               'к': 12,
               'л': 13,
               'м': 14,
               'н': 15,
               'о': 16,
               'п': 17,
               'р': 18,
               'с': 19,
               'т': 20,
               'у': 21,
               'ф': 22,
               'х': 23,
               'ц': 24,
               'ч': 25,
               'ш': 26,
               'щ': 27,
               'ъ': 28,
               'ы': 29,
               'ь': 30,
               'э': 31,
               'ю': 32,
               'я': 33}

inv_mapping = dict(zip(vocabulary.values(), vocabulary.keys()))
inv_mapping[34]='<пробел>'

In [3]:
def decode_single(session, test_input):
    z=np.zeros((30,13))
    zz=np.vstack((test_input,z))

    val_feed = {
        input_X:  np.asarray([zz]),
        seq_lens: np.asarray([len(test_input)])
    }

    # Decoding
    d = session.run(decoded[0], feed_dict=val_feed)
    dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=session)

    seq = [s for s in dense_decoded[0] if s != -1]
    ret=decode(d, inv_mapping )
    for i in range(len(ret)):
            print(str(ret[i])),
    print('')
    

In [4]:
def decode(d, mapping):
    """Decode."""
    shape = d.dense_shape
    batch_size = shape[0]
    ans = np.zeros(shape=shape, dtype=int)
    seq_lengths = np.zeros(shape=(batch_size, ), dtype=np.int)
    for ind, val in zip(d.indices, d.values):
        ans[ind[0], ind[1]] = val
        seq_lengths[ind[0]] = max(seq_lengths[ind[0]], ind[1] + 1)
    ret = []
    for i in range(batch_size):
        ret.append("".join(map(lambda s: mapping[s], ans[i, :seq_lengths[i]])))
    return ret


# модель

In [5]:
graph = tf.Graph()
with graph.as_default():
    input_X = tf.placeholder(tf.float32, shape=[None, None, 13],name="input_X")
    labels = tf.sparse_placeholder(tf.int32)
    seq_lens = tf.placeholder(tf.int32, shape=[None],name="seq_lens")


    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True, implementation=2), input_shape=(None, 13)))
    model.add(Bidirectional(LSTM(128, return_sequences=True, implementation=2)))
    model.add(TimeDistributed(Dense(len(inv_mapping) + 2)))
    
    final_seq_lens = seq_lens

    logits = model(input_X)
    logits = tf.transpose(logits, [1, 0, 2])

    ctc_loss = tf.reduce_mean(tf.nn.ctc_loss(labels, logits, final_seq_lens))
    # ctc_greedy_decoder? merge_repeated=True
    decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, final_seq_lens)
    ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32), labels))

    train_op = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(ctc_loss)


Скачиваем   тестовый wav фаил с мужским голосом

In [6]:
WAVE_OUTPUT_FILENAME =  'test.wav'

In [7]:
sample_rate, X1=  wav.read(WAVE_OUTPUT_FILENAME)
# Через несколько лет путешествие на Марс будет не более сложно, чем перелёт, из Москвы в Берлин. 
Audio(data=X1, rate=sample_rate)

Выдиляем из файла фичи MFCC

In [8]:
fs, audio = wav.read(WAVE_OUTPUT_FILENAME)

features = mfcc(audio, samplerate=fs, lowfreq=50)

mean_scale = np.mean(features, axis=0)
std_scale = np.std(features, axis=0)

features = (features - mean_scale[np.newaxis, :]) / std_scale[np.newaxis, :]

seq_len = features.shape[0]


 Распознаем речь  на предворительно обученной модели

In [9]:
with tf.Session(graph=graph) as session:

    saver = tf.train.Saver(tf.global_variables())
    snapshot = "ctc"
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint1")
    last_epoch = 0

    if checkpoint:
        print("[i] LOADING checkpoint " + checkpoint)
        try:
            saver.restore(session, checkpoint)
           
        except:
            print("[!] incompatible checkpoint, restarting from 0")
    else:
        # Initializate the weights and biases
        tf.global_variables_initializer().run()
    decode_single(session, features)   

[i] LOADING checkpoint checkpoint1/ctc.ckpt-699
INFO:tensorflow:Restoring parameters from checkpoint1/ctc.ckpt-699
<пробел>через<пробел>несколько<пробел>лет<пробел>путешествие<пробел>на<пробел>марс<пробел>будет<пробел>не<пробел>более<пробел>сложно<пробел>чем<пробел>перелёт<пробел>из<пробел>москвы<пробел>в<пробел>берлин<пробел> 


# Тест распознования женского голоса, говорящего на русском языке

In [10]:
WAVE_OUTPUT_FILENAME =  'ru_test.wav'
# Покалывало грудь стучала кровь в виски но дышалось легко воздух был тонок и сух
sample_rate, X1=  wav.read(WAVE_OUTPUT_FILENAME)
# Через несколько лет путешествие на Марс будет не более сложно, чем перелёт, из Москвы в Берлин. 
Audio(data=X1, rate=sample_rate)

In [11]:
fs, audio = wav.read(WAVE_OUTPUT_FILENAME)

features = mfcc(audio, samplerate=fs, lowfreq=50)

mean_scale = np.mean(features, axis=0)
std_scale = np.std(features, axis=0)

features = (features - mean_scale[np.newaxis, :]) / std_scale[np.newaxis, :]

seq_len = features.shape[0]
with tf.Session(graph=graph) as session:

    saver = tf.train.Saver(tf.global_variables())
    snapshot = "ctc"
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint1")
    last_epoch = 0

    if checkpoint:
        print("[i] LOADING checkpoint " + checkpoint)
        try:
            saver.restore(session, checkpoint)
            
        except:
            print("[!] incompatible checkpoint, restarting from 0")
    else:
        # Initializate the weights and biases
        tf.global_variables_initializer().run()
    decode_single(session, features)   

[i] LOADING checkpoint checkpoint1/ctc.ckpt-699
INFO:tensorflow:Restoring parameters from checkpoint1/ctc.ckpt-699
<пробел>покалывало<пробел>грудь<пробел>стучала<пробел>кровь<пробел>в<пробел>виски<пробел>но<пробел>дышалось<пробел>легко<пробел>воздух<пробел>был<пробел>тонок<пробел>и<пробел>сух<пробел> 


# Тест акустической модели с микрофона

In [12]:
import pyaudio
import wave
# and IPython.display for audio output
import IPython.display
from scipy.io import wavfile


In [13]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 3 #время записи
WAVE_OUTPUT_FILENAME =  'mikr.wav'

запись с микрофона в wav

In [14]:
p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* ЗАПИСЬ С МИКРОФОНА")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* КОНЕЦ ЗАПИСИ")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()
fs, audio = wav.read(WAVE_OUTPUT_FILENAME)

features = mfcc(audio, samplerate=fs, lowfreq=50)

mean_scale = np.mean(features, axis=0)
std_scale = np.std(features, axis=0)

features = (features - mean_scale[np.newaxis, :]) / std_scale[np.newaxis, :]

seq_len = features.shape[0]


* ЗАПИСЬ С МИКРОФОНА
* КОНЕЦ ЗАПИСИ


In [15]:
sample_rate, X1=  wavfile.read(WAVE_OUTPUT_FILENAME)
# Play it back!
IPython.display.Audio(data=X1, rate=sample_rate)

In [16]:
with tf.Session(graph=graph) as session:

    saver = tf.train.Saver(tf.global_variables())
    snapshot = "ctc"
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint1")
    last_epoch = 0

    if checkpoint:
        print("[i] LOADING checkpoint " + checkpoint)
        try:
            saver.restore(session, checkpoint)
            
        except:
            print("[!] incompatible checkpoint, restarting from 0")
    else:
        # Initializate the weights and biases
        tf.global_variables_initializer().run()
    decode_single(session, features)   

[i] LOADING checkpoint checkpoint1/ctc.ckpt-699
INFO:tensorflow:Restoring parameters from checkpoint1/ctc.ckpt-699
<пробел>в<пробел>ключих<пробел>свет<пробел>в<пробел>гростинной<пробел> 
